# DIS08 - Web Scraping

The following Jupyter notebook is based on chapter 11 of [Automatic the Boring Stuff with Python](https://automatetheboringstuff.com) and the tutorials of [The Library Carpentry](https://librarycarpentry.org/lc-webscraping).

## What is web scraping?

Web scraping is a technique for extracting information from websites. This can be done *manually* but it is usually faster, more efficient and less error-prone to *automate* the task.

Web scraping allows you to acquire non-tabular or poorly structured data from websites and convert it into a usable, structured format, such as a .csv file or spreadsheet.

Scraping is about more than just acquiring data: it can also help you archive data and track changes to data online.

It is closely related to the practice of *web indexing*, which is what search engines like Google do when mass-analysing the Web to build their indices. But contrary to web indexing, which typically parses the entire content of a web page to make it searchable, web scraping targets specific information on the pages visited. **You already learned something on this practice in my lecture on Information Retrieval last semester.**

For example, online stores will often scour the publicly available pages of their competitors, scrape item prices, and then use this information to adjust their own prices. Another common practice is “contact scraping” in which personal information like email addresses or phone numbers is collected for marketing purposes.

Web scraping is also increasingly being used by scholars to create data sets for text mining projects; these might be collections of journal articles or digitised texts. The practice of data journalism, in particular, relies on the ability of investigative journalists to harvest data that is not always presented or published in a form that allows analysis.

## Before you get started

As useful as scraping is, there might be better options for the task. Choose the right (i.e. the easiest) tool for the job.

* Check whether or not you can easily *copy and paste data* from a site into Excel or Google Sheets. This might be quicker than scraping.
* Check if the site or service already provides an *API* to extract structured data. If it does, that will be a much more efficient and effective pathway. Good examples are the Facebook API, the Twitter APIs or the YouTube comments API.
* For much larger needs, *Freedom of information requests* (in case you are in the US) can be useful. Be specific about the formats required for the data you want.

## Structured vs unstructured data

When presented with information, human beings are good at quickly categorizing it and extracting the data that they are interested in. For example, when we look at a magazine rack, provided the titles are written in a script that we are able to read, we can rapidly figure out the titles of the magazines, the stories they contain, the language they are written in, etc. and we can probably also easily organize them by topic, recognize those that are aimed at children, or even whether they lean toward a particular end of the political spectrum. Computers have a much harder time making sense of such unstructured data unless we specifically tell them what elements data is made of, for example by adding labels such as this is the title of this magazine or this is a magazine about food. Data in which individual elements are separated and labelled is said to be structured.

Let’s look now at the current list of members for the [UK House of Commons](https://www.parliament.uk/mps-lords-and-offices/mps/).

![](https://librarycarpentry.org/lc-webscraping/fig/ukparl.png)

This page also displays a list of names, political and geographical affiliation. There is a search box and a filter option, but no obvious way to download this information and reuse it.

Here is the code for this page:

```html
(...)
<table>
    <tbody>
        (...)
        <tr id="ctl00_ctl00_(...)_trItemRow" class="first">
            <td>Aberavon</td>
            <td id="ctl00_ctl00_(...)_tdNameCellRight">
                <a id="ctl00_ctl00_(...)_hypName" href="http://www.parliament.uk/biographies/commons/stephen-kinnock/4359">Kinnock, Stephen</a>(Labour)
            </td>
        </tr>
        (...)
    </tbody>
</table>
(...)
```

We see that this data has been structured for displaying purposes (it is arranged in rows inside a table) but the different elements of information are not clearly labelled.

What if we wanted to download this dataset and, for example, compare it with the German list of members of the Bundestag to analyze gender representation, or the representation of political forces in the two groups? We could try copy-pasting the entire table into a spreadsheet or even manually copy-pasting the names and parties in another document, but this can quickly become impractical when faced with a large set of data. What if we wanted to collect this information for every country that has a parliamentary system?

Fortunately, there are tools to automate at least part of the process. This technique is called web scraping.

```
Web scraping (web harvesting or web data extraction) is a computer software technique of extracting information from websites. (Source: Wikipedia)
```
    
Web scraping typically targets one web site at a time to extract unstructured information and put it in a structured form for reuse.

In this lesson, we will continue exploring the examples above and try different techniques to extract the information they contain. But before we launch into web scraping proper, we need to look a bit closer at how information is organized within an HTML document and how to build queries to access a specific subset of that information.

## Webscraping with Beautiful Soup

Beautiful Soup is a module for extracting information from an HTML page (and is much better for this purpose than regular expressions). The BeautifulSoup module’s name is `bs4` (for Beautiful Soup, version 4). To install it, you will need to run `pip install beautifulsoup4` or `pip3 install beautifulsoup4` from the command line. 

(Check out [the appendix from Automate the Boring Stuff](https://automatetheboringstuff.com/appendixa/) for instructions on installing third-party modules.) While `beautifulsoup4` is the name used for installation, to import Beautiful Soup you run `import bs4`.

In [ ]:
import bs4
import requests

resource = requests.get('http://www.th-koeln.de')
thSoup = bs4.BeautifulSoup(resource.content)
type(thSoup)


This code uses `requests.get()` to download the main page from the TH Köln website and then passes the text attribute of the response to `bs4.BeautifulSoup()`. The BeautifulSoup object that it returns is stored in a variable named `thSoup`.

Once you have a BeautifulSoup object, you can use its methods to locate specific parts of an HTML document.

## Finding an Element with the select() Method

You can retrieve a web page element from a BeautifulSoup object by calling the `select()` method and passing a string of a CSS *selector* for the element you are looking for. Selectors are like *regular expressions*: They specify a pattern to look for, in this case, in HTML pages instead of general text strings.

Here is an overview on some common CSS selectors:

|Selector passed to the select() method | Will match...|
|-------|-------|
|`soup.select('div')`|All elements named `<div>`|
|`soup.select('#author')`|The element with an __id attribute__ of author|
|`soup.select('.notice')`|All elements that use a CSS __class attribute__ named notice|
|`soup.select('div span')`|All elements named `<span>` that are within an element named `<div>`|
|`soup.select('div > span')`|All elements named `<span>` that are directly within an element named `<div>`, with no other element in between |
|`soup.select('input[name]')`|All elements named `<input>` that have a name attribute with any value|
|`soup.select('input[type="button"]')`|All elements named `<input>` that have an attribute named type with value button|

The various selector patterns can be combined to make sophisticated matches. For example, `soup.select('p #author')` will match any element that has an `id` attribute of author, as long as it is also inside a `<p>` element.

The `select()` method will return a list of `Tag` objects, which is how Beautiful Soup represents an HTML element. The list will contain one `Tag` object for every match in the BeautifulSoup object’s HTML. `Tag` values can be passed to the `str()` function to show the HTML tags they represent.

In [ ]:
import bs4
import requests

# get the HTML resource from the web
resource = requests.get('http://www.zeit.de')
# let Beautiful Soup parse the ressource
zeitSoup = bs4.BeautifulSoup(resource.content)
# select all <span> with the class "zon-teaser-standard__title"
elements = zeitSoup.select('span.zon-teaser-standard__title')
# What type are the parsed elements
print(type(elements))
print(len(elements))
print(elements)

In [ ]:
for element in elements:
    if(element.string):
        print(element.string)

## Getting Data from an Element’s Attributes

`Tag` values also have an `attrs` attribute that shows all the HTML attributes of the tag as a dictionary. The `get()` method for Tag objects makes it simple to access attribute values from an element. The method is passed a string of an attribute name and returns that attribute’s value.

In [ ]:
testHTML = '''<span type='dummy' content='none'>lorem ipsum</span>'''
# you can also read in files or simple strings
exampleSoup = bs4.BeautifulSoup(testHTML)
# This is an alternative way to access a Tag object
# list all attributes of a Tag objects
print(exampleSoup.span.attrs)
# access a 
print(exampleSoup.span.attrs.get('type'))


In [ ]:
for image in zeitSoup.select('img'):
    print(image.attrs.get('src'))


## Beautiful Soup and RegExs

Of course you can mix Beautiful Soup with RegEx. 

Beautiful Soup defines a lot of methods for searching the parse tree, but they’re all very similar. I’m going to spend a lot of time explaining the two most popular methods: `find()` and `find_all()`. 

The simplest filter is a string. Pass a string to a search method and Beautiful Soup will perform a match against that exact string. This code finds all the `<a>` tags in the document:

In [ ]:
thSoup.find_all('a')

If you pass in a regular expression object, Beautiful Soup will filter against that regular expression. Here we will use a RegEx on the strings within the tags, not the tag names themself.

In [ ]:
import re

# return all strings that start with TH
thSoup.find_all(string=re.compile("^TH"))


## Exercise 3 - Scrape the Wikipedia (25 pts) 

In this final assignment, we would like you to develop a little web scraping project. This is the last part of the second assignment for this semester. It includes a lot of the different tools and skills you learned during this semester.

1. Pick a list within the Wikipedia like the [list of sovereign states](https://en.wikipedia.org/wiki/List_of_sovereign_states). Choose some other list on your own, based on your personal interests. The only requirement is that there are other Wikipedia articles linked within the list. If you have taken this course in the past, make sure to choose a different list!
2. Get all the names and URLs to the corresponding items in the list and export them into a CSV file that has two columns (name and URL).
3. For every Wikipedia article in the CSV list choose a few attributes from the infobox on the right that you would like to extract (e.g., population, name of the head of state, whatever...). These attributes should have at least a little bit in common with the "source list". Extract this information for every entry in your list. Store this information in an appropriate data structure. Make sure to clean your scraped data if necessary!
4. Save your scraped information into a JSON file. Try to export *clean* data.
5. Document your program and development process (in a markdown file). Tell us something about the data you scraped. Why did you choose this data? Can you think of a good use case for this data? As always: Name your files and push everything into your GitHub repository.



### Some hints

* Try to be kind to Wikipedia and yourself. You will most likely generate a lot of web traffic while scraping the same webpage again and again. This stresses the Wikipedia's server and takes a lot of time. Try to use a caching method like the one from [requests-cache](https://pypi.org/project/requests-cache/). Alternatively, you can download the HTML content using your script and then work locally.
* Try not to solve the whole problem at once. Remember the tactics desribed in the earlier lectures: [Divide and conquer](https://en.wikipedia.org/wiki/Divide-and-conquer_algorithm) - Step by step. 
* Have a look at the two sample projects from [chapter 11](https://automatetheboringstuff.com/chapter11/). They do something similar.
* A lot of code examples for Beautiful Soup are documented in the [official documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).
* Again: Documentation is key! Everything that is **not properly documented** is not verifyable by us and will thus get **0 points**. 

